In [7]:
from google.colab import auth

# Aunthenticate to Google Colab
auth.authenticate_user()

#Configure Google Cloud
from tools import load_db_cfg
dbcfg=load_db_cfg('pp.json')
!gcloud config set project {dbcfg['project_id']}

'''
  Enable Google SQL Admin API Services
'''
# Enable Cloud SQL Admin API
!gcloud services enable sqladmin.googleapis.com

Updated property [core/project].


In [8]:
!pip install cloud-sql-python-connector

In [9]:
'''
  Drop DB if exists and create it afresh
'''
from tools import load_db_cfg
dbcfg=load_db_cfg('pp.json')
db_name=dbcfg['name']
instance_name = dbcfg['instance_name']
!gcloud sql databases delete {db_name} --instance={instance_name}

print(f'''Database '{db_name}' deleted successfully !!''')


The database will be deleted. Any data stored in the database will be destroyed.
 You cannot undo this action.

Do you want to continue (Y/n)?  Y

Deleted database [phonepe_pulse].
Database 'phonepe_pulse' deleted successfully !!


In [10]:
!gcloud sql databases create {db_name} --instance={instance_name}
print(f'''Database '{db_name}' created successfully !!''')


Created database [phonepe_pulse].
instance: mysql-hg1
name: phonepe_pulse
project: youtubechannelanalysis-423709
Database 'phonepe_pulse' created successfully !!


In [11]:
# Install PyMySQL python module
!pip install pymysql

In [12]:
'''
  Connect to Cloud SQL DB to create the schema for 'phonepe' database
'''
from tools import load_db_cfg
from dbconnect import DbConnector
dbcfg=load_db_cfg('pp.json')
pool = DbConnector(dbcfg)

'''
  Create the required tables for 'phonepe' database
'''
import sqlalchemy
with pool.connect() as db:
  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS geo (
        id INT NOT NULL,
        geo_type ENUM('CON','STA','DIS','CIT','PIN'),
        name VARCHAR(255) NOT NULL,
        latitude INT,
        longitude INT,
        parent_id INT,
        PRIMARY KEY (id),
        INDEX name (name));'''))

  db.execute(
    sqlalchemy.text(
      '''CREATE TABLE IF NOT EXISTS transaction_agg (
        id INT NOT NULL,
        year INT NOT NULL,
        quarter INT NOT NULL,
        geo_id INT NOT NULL,
        category VARCHAR(255) NOT NULL,
        stat_type VARCHAR(50) NOT NULL,
        count BIGINT NOT NULL DEFAULT 0,
        amount FLOAT NOT NULL DEFAULT 0,
        PRIMARY KEY (id),
        CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,category),
        INDEX rec_index (year,quarter,geo_id),
        FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS transaction_top (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          top_in ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id, top_in),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS transaction_hover (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          hover_over ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS insurance_agg (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          category VARCHAR(255) NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,category),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS insurance_top (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          top_in ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          stat_type VARCHAR(50) NOT NULL,
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,top_in),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
      sqlalchemy.text(
          '''CREATE TABLE IF NOT EXISTS insurance_hover (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          hover_over ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          count BIGINT NOT NULL DEFAULT 0,
          amount FLOAT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS user_agg (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          reg_users BIGINT NOT NULL DEFAULT 0,
          app_opens BIGINT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS device_agg (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          brand VARCHAR(255) NOT NULL,
          count BIGINT NOT NULL,
          percentage INT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,brand),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS user_top (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          top_in ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          stat_type VARCHAR(50) NOT NULL,
          reg_users BIGINT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id,top_in),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

  db.execute(
    sqlalchemy.text('''CREATE TABLE IF NOT EXISTS user_hover (
          id INT NOT NULL,
          year INT NOT NULL,
          quarter INT NOT NULL,
          geo_id INT NOT NULL,
          stat_type VARCHAR(50) NOT NULL,
          hover_over ENUM('CON','STA') NOT NULL DEFAULT 'CON',
          reg_users BIGINT NOT NULL DEFAULT 0,
          app_opens BIGINT NOT NULL DEFAULT 0,
          PRIMARY KEY (id),
          CONSTRAINT rec_unique UNIQUE(year,quarter,geo_id),
          INDEX rec_index (year,quarter,geo_id),
          FOREIGN KEY (geo_id) REFERENCES geo(id) ON DELETE CASCADE);'''))

# Commit the above changes to DB
db.commit()
print( f'''Tables for '{db_name}' database successfully created !! ''')
db.close()

Successfully connected to 'phonepe_pulse' database!! 
Tables for 'phonepe_pulse' database successfully created !! 


In [ ]:
from tools import load_db_cfg
from dbconnect import DbConnector
from pp_domain import Geo
dbcfg=load_db_cfg('pp.json')
pool = DbConnector(dbcfg)
con=pool.connect()
geo=Geo()
geo.from_csv('/content')
print (f'Geo: states={geo.states().shape}, districts={geo.districts().shape}, pincodes={geo.pincodes().shape}')
geo.store(con)
con.close()

Successfully connected to 'phonepe' database!! 
Geo: states=(36, 2), districts=(417, 2), pincodes=(1115, 2)
